# Flux balance analysis (FBA)

We will use `flux_balance_analysis` and several related functions to find the optimal flux in the *E. coli* "core" model.

If it is not already present, download the model and load the package:

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel
sparse([41, 23, 51, 67, 61, 65, 1, 7, 19, 28  …  72, 3, 8, 33, 57, 66, 31, 45, 46, 57], [1, 2, 2, 2, 3, 3, 4, 4, 4, 4  …  93, 94, 94, 94, 94, 94, 95, 95, 95, 95], [-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0  …  1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


To perform any optimization-based analysis, we need to use a linear programming
solver (also called an optimizer). Any of the [`JuMP.jl`-supported
optimizers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers)
will work. Here, we will demonstrate
[`Tulip.jl`](https://github.com/ds4dm/Tulip.jl) and
[GLPK](https://www.gnu.org/software/glpk/); other solvers will likely work just as
well.

In [2]:
using Tulip

solved_model = flux_balance_analysis(model, Tulip.Optimizer)

A JuMP Model
Maximization problem with:
Variables: 95
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 72 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 190 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Tulip
Names registered in the model: lbs, mb, ubs, x

`solved_model` is now an instance of optimized JuMP model. To get the
variable values out manually, we can use `JuMP.value` function. Flux variables
are stored as vector `x`:

In [3]:
using JuMP
value.(solved_model[:x])

95-element Vector{Float64}:
  -0.0
   6.00724956649032
   7.477381918907127
  -5.064375360152338
   0.2234617471432185
  -3.214895030387032
   2.504309432010867
  21.799492758475754
   4.959985078874371
   1.496983802869297
   ⋮
   3.375438217960911e-7
  29.175827202685298
   9.054357964341115e-9
   4.817965631705414e-8
   9.959461594581987e-9
 -21.799492758475754
  -0.0
  -1.4340676616267298e-9
   3.214895030387032

To simplify things, there is a variant of the FBA function that does this for
us automatically:

In [4]:
flux_balance_analysis_vec(model, Tulip.Optimizer)

95-element Vector{Float64}:
   0.0
   6.00724956649032
   7.477381918907127
  -5.064375360152338
   0.2234617471432185
  -3.214895030387032
   2.504309432010867
  21.799492758475754
   4.959985078874371
   1.496983802869297
   ⋮
   3.375438217960911e-7
  29.175827202685298
   9.054357964341115e-9
   4.817965631705414e-8
   9.959461594581987e-9
 -21.799492758475754
   0.0
  -1.4340676616267298e-9
   3.214895030387032

Likewise, there is another variant that returns the fluxes annotated by
reaction names, in a dictionary:

In [5]:
flux_balance_analysis_dict(model, Tulip.Optimizer)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => 0.0
  "R_ACONTb"      => 6.00725
  "R_TPI"         => 7.47738
  "R_SUCOAS"      => -5.06438
  "R_GLNS"        => 0.223462
  "R_EX_pi_e"     => -3.2149
  "R_PPC"         => 2.50431
  "R_O2t"         => 21.7995
  "R_G6PDH2r"     => 4.95999
  "R_TALA"        => 1.49698
  "R_PPCK"        => 5.88317e-8
  "R_EX_lac__D_e" => 2.39394e-9
  "R_PGL"         => 4.95999
  "R_H2Ot"        => -29.1758
  "R_GLNabc"      => 0.0
  "R_EX_co2_e"    => 22.8098
  "R_EX_gln__L_e" => 0.0
  "R_EX_nh4_e"    => -4.76532
  "R_MALt2_2"     => 0.0
  ⋮               => ⋮

Switching solvers is easy, and may be useful in case we need advanced
functionality or performance present only in certain solvers. To switch to
GLPK, we simply load the package and use a different optimizer to run the
analysis:

In [6]:
using GLPK
flux_balance_analysis_dict(model, GLPK.Optimizer)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => 0.0
  "R_ACONTb"      => 6.00725
  "R_TPI"         => 7.47738
  "R_SUCOAS"      => -5.06438
  "R_GLNS"        => 0.223462
  "R_EX_pi_e"     => -3.2149
  "R_PPC"         => 2.50431
  "R_O2t"         => 21.7995
  "R_G6PDH2r"     => 4.95998
  "R_TALA"        => 1.49698
  "R_PPCK"        => -9.9537e-15
  "R_EX_lac__D_e" => -2.05011e-14
  "R_PGL"         => 4.95998
  "R_H2Ot"        => -29.1758
  "R_GLNabc"      => 0.0
  "R_EX_co2_e"    => 22.8098
  "R_EX_gln__L_e" => 0.0
  "R_EX_nh4_e"    => -4.76532
  "R_MALt2_2"     => -1.30217e-14
  ⋮               => ⋮

To get a shortened but useful overview of what was found in the analysis, you
can use `flux_summary` function:

In [7]:
flux_summary(flux_balance_analysis_dict(model, GLPK.Optimizer))

Biomass
  R_BIOMASS_Ecoli_core_w_GAM: 0.8739
Import
  R_EX_o2_e:                  -21.7995
  R_EX_glc__D_e:              -10.0
  R_EX_nh4_e:                 -4.7653
  R_EX_pi_e:                  -3.2149
Export
  R_EX_h_e:                   17.5309
  R_EX_co2_e:                 22.8098
  R_EX_h2o_e:                 29.1758


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*